In [ ]:
# @author Neill Kaipo Shikada
## ATLAS Institute
## Artist Statement
    ## Create a cooperative drawing program meant to use pose detection to allow drawing between an
    ## participant's hands. The distance between their hands dictates the radius while the mean between
    ## the hands is the centerpoint of the circle.
    ## This refactored version uses object oriented programming to create a list of cirlcle objects
    
    ## Code inspired by Nicholas Renotte

In [ ]:
!pip install mediapipe opencv-python

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import uuid
import os
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [2]:
def calculate_mean(a,b):
    a = np.array(a) #First point
    b = np.array(b) #Second point
    
    # Calculate mean of the points
    x = (a[0]+b[0])/2
    y = (a[1]+b[1])/2
    
    return x,y

In [3]:
def calculate_distance(a,b):
    a = np.array(a) #First point
    b = np.array(b) #Second point
    x1 = a[0]
    x2 = b[0]
    y1 = a[1]
    y2 = b[1]
    
    # Calculate the distance between the points
    d = ((((x2 - x1 )**2) + ((y2-y1)**2) )**0.5)
    
    return d

In [4]:
def calculate_depth(z):
    
    depth = (z*-100) + 100
    
    return depth

In [8]:
# Object oriented format for circles to draw themselves    
class Circle:
    
    #def getColor(self):
    #    color = ((self.x/2), (self.depth*0.5), (self.depth*1.25))
    #    return color
    
    def __init__(self, x, y, radius, depth):
        self.x = x
        self.y = y
        self.radius = radius
        self.depth = depth
    #   self.color = getColor(self)
        self.color = (self.depth, self.depth, self.depth)
        
    def draw(self):
        cv2.circle(image, (self.x,self.y), self.radius, self.color, 1)

In [9]:
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise Exception("Could not open video device")
# Adjust the window dimensions
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1080)

# Make Window fullscreen
window_name = "Cooperative Forms"
cv2.namedWindow(window_name, cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty(window_name, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

circleList = []

## Setup the mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.8, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Image
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # Flip image horizontally
        image = cv2.flip(image, 1)
        image.flags.writeable = False
        
        # Activates the pose detection function from mediapipe and stores those in an array in "results"
        results = pose.process(image)
        
        # Recolors image back
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Extract Landmarks
            ri = [landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value].y]
            li = [landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].x,landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].y]
            rz = landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value].z
            lz = landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].z
            rviz = landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value].visibility
            lviz = landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].visibility
            
            # Calculate Mean
            mean = calculate_mean(ri, li)
            distance = calculate_distance(ri, li)
            
        except:
            pass
        
        cv2.rectangle(image, (0,0), (1920,1080), (0,0,0), -1)
        
        try:
            x = int(mean[0]*960)
            y = int(mean[1]*480)
            radius = int((distance*15)**2)
            depth = calculate_depth(rz)
            
            if rviz >= 0.5 and lviz >= 0.5:
                if radius <= 100:
                    circleList.append(Circle(x,y,radius,depth))
                    
            i = 0
            while i<len(circleList):
                for i in circleList:
                    Circle.draw(i)
                i += 1
        except:
            pass
        
        if cv2.waitKey(10) & 0xFF == ord(' '):
            circleList = []
        
        # Render the detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(150,150,150), thickness=1, circle_radius=2),
                                mp_drawing.DrawingSpec(color=(150,150,150), thickness=1, circle_radius=2)
                                )
        
        cv2.imshow('Cooperative Forms', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

# Save final frame as an image 
cv2.imwrite(os.path.join('Output Images', '{}.jpg'.format(uuid.uuid1())), image)
cv2.imshow('Cooperative Forms', image)   

cap.release()
cv2.destroyAllWindows()

In [7]:
cap.release()
cv2.destroyAllWindows()